In [1]:
import tensorflow as tf
import nsvision as nv
import requests
import json
import numpy as np
print(tf.__version__)

2.3.0


In [2]:
path_saved_model = 'flowerclassifier/001'

In [3]:
working_model = tf.saved_model.load(path_saved_model)

In [4]:
working_model.signatures.keys()

KeysView(_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(input_4) at 0x7FD648395FD0>}))

In [5]:
print(list(working_model.signatures.keys())) 

['serving_default']


The model, once it's loaded in memory, will have to be an `infer` object, so it can make a prediction output with an input data.

In [6]:
infer = working_model.signatures['serving_default']

The model output signature tells us the data structure of the output.

In [7]:
print(infer.structured_outputs)

{'custom_class': TensorSpec(shape=(None, 5), dtype=tf.float32, name='custom_class')}


The output is an array of five floating point values. The output node name is `custom_class`. Next, we will use the following image as our input image to the model for scoring:

In [9]:
jpg1 = 'raw_images/2440874162_27a7030402_n.jpg' # it is a tulip

In [10]:
img1_np = nv.imread(jpg1, resize=(224,224),normalize=True) #Width and height required. RGB dimension preserved. Numpy.
#img2_np = nv.imread(jpg2, resize=(224,224),normalize=True)
img1_np = nv.expand_dims(img1_np,axis=0)

A prediction is made for flower image `img1_np`.

In [11]:
prediction = infer(tf.constant(img1_np))

In [12]:
prediction['custom_class'].numpy()

array([[2.4262092e-06, 5.6151916e-06, 1.8000206e-05, 1.4342861e-05,
        9.9995959e-01]], dtype=float32)

Now we will reuse the label map from Chapter 7's exercises:

In [13]:
val_label_map = {4: 'tulips', 3: 'dandelion', 1: 'sunflowers', 2: 'daisy', 0: 'roses'}

In [14]:
val_label_map

{4: 'tulips', 3: 'dandelion', 1: 'sunflowers', 2: 'daisy', 0: 'roses'}

In [15]:
def lookup(np_entry, dictionary):
    class_key = np.argmax(np_entry)
    return dictionary.get(class_key)

In [16]:
lookup(prediction['custom_class'].numpy(), val_label_map)

'tulips'

## Build JSON Payload

In [17]:
data = json.dumps({ 
    "instances": img1_np.tolist()
})
headers = {"content-type": "application/json"}

In [18]:
 data

'{"instances": [[[[0.10588235408067703, 0.16078431904315948, 0.0313725508749485], [0.1764705926179886, 0.24313725531101227, 0.07058823853731155], [0.1725490242242813, 0.27843138575553894, 0.01568627543747425], [0.8274509906768799, 0.4588235318660736, 0.5647059082984924], [0.8705882430076599, 0.4274509847164154, 0.5372549295425415], [0.8039215803146362, 0.47843137383461, 0.4156862795352936], [0.8627451062202454, 0.38823530077934265, 0.4470588266849518], [0.7803921699523926, 0.2862745225429535, 0.3686274588108063], [0.8549019694328308, 0.40392157435417175, 0.45490196347236633], [0.8392156958580017, 0.35686275362968445, 0.3764705955982208], [0.8392156958580017, 0.27450981736183167, 0.2705882489681244], [0.8705882430076599, 0.30980393290519714, 0.24705882370471954], [0.8470588326454163, 0.3490196168422699, 0.2823529541492462], [0.843137264251709, 0.5176470875740051, 0.48627451062202454], [0.8745098114013672, 0.4901960790157318, 0.47843137383461], [0.7333333492279053, 0.3529411852359772, 0.

In [19]:
response = requests.post('http://localhost:8501/v1/models/flowerclassifier:predict', data=data, headers=headers)

In [20]:
response.json()

{'predictions': [[2.42621149e-06,
   5.61519164e-06,
   1.80002226e-05,
   1.43428879e-05,
   0.999959588]]}

Use `lookup` to decode response from the model:

In [21]:
lookup(response.json()['predictions'], val_label_map)

'tulips'